<a href="https://colab.research.google.com/github/dlsun/Data402-F21/blob/main/Multiclass_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

We have looked at the MNIST handwritten digits data set before. Previously, we classified whether a given digit was 5 or not (binary classification). Now that we know how to do multiclass classification, let's treat this properly as a multiclass classification function with 10 classes.

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

Notice that the pixel values are 8-bit integers, but the input to neural networks needs to be floats. The code below normalizes the images so that the values are floats.

In [ ]:
x_train_float = x_train / 255
x_test_float = x_test / 255

We also need to convert the label into a vector of 0s and 1s.

In [ ]:
y_train_enc = tf.keras.utils.to_categorical(y_train)
y_test_enc = tf.keras.utils.to_categorical(y_test)

y_train_enc

Now, we will fit a standard convolutional neural network (CNN) architecture called [LeNet](https://en.wikipedia.org/wiki/LeNet) to this data. This architecture was developed by Yann LeCun in 1989 for this specific data set.

The very first layer of this network is a 2D convolution. We can use the Keras layer `Conv2D`. If you look at the [documentation](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D), `Conv2D` expects a 4-D tensor as input.

Right now, our input `x_train` and `x_test` only has 3 dimensions.

- 1 dimension for the examples
- 1 dimension for the height
- 1 dimension for the width

What is the 4th dimension? We need to create a dummy 4th dimension.

In [ ]:
x_train_float = x_train_float[:, :, :, np.newaxis]
x_test_float = x_test_float[:, :, :, np.newaxis]

x_train_float.shape

Now we are ready to define the model. A convolutional neural network usually begins with alternating convolutional and pooling layers.

To do the convolution, we use [`Conv2D`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D) in Keras. LeNet uses average pooling, which is [`AvgPool2D`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/AvgPool2D) in Keras.

In [ ]:
model = tf.keras.models.Sequential([
  layers.Conv2D(filters=6, kernel_size=(5, 5), strides=(1, 1), padding="same",
                activation="sigmoid"),
  layers.AvgPool2D(pool_size=(2, 2), strides=(2, 2)),
  layers.Conv2D(filters=16, kernel_size=(5, 5), strides=(1, 1), padding="valid",
                activation="sigmoid"),
  layers.AvgPool2D(pool_size=(2, 2), strides=(2, 2)),
  layers.Flatten(),
  layers.Dense(120, activation="sigmoid"),
  layers.Dense(84, activation="sigmoid"),
  layers.Dense(10, activation="softmax")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=["accuracy"],
)

Now we are ready to fit the model to data.

In [ ]:
model.fit(x_train_float, y_train_enc,
          epochs=3,
          batch_size=30,
          validation_data=(x_test_float, y_test_enc))

You can print out a description of your model, which can help you debug your network.

In [ ]:
model.summary()